In [1]:
import pandas as pd
import numpy as np
import sqlite3
import warnings
import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
#Check requirements
if pd.__version__ != '1.4.2':
    print('Warning: using Pandas version other than 1.4.2')
elif np.__version__ != '1.22.3':
    print('Warning: using NumPy version other than 1.22.3')
elif sqlite3.sqlite_version != '3.36.0':
    print('Warning: using SQLite3 version other than 3.36.0')
elif tqdm.__version__ != '4.62.3':
    print('Warning: using tqdm version other than 4.62.3')
else:
    print('No package conflicts')

# Read data and insert into SQL Lite

### 1. Reliability data

In [2]:
rel_Dat = pd.read_excel('data_house.xlsx', sheet_name='Reliability Data', header = 2) #read in data

Clean the order data. This step might take up to 20 minutes.

In [3]:
clean = [pd.to_datetime('2021-07-20', format="%Y-%m-%d")]*23
rel_Dat['PO Document Date'][3:26]=clean #manually fix bad data
def data_changer(row): #create function to fix date times
    date = row['PO Document Date']
    if type(date) != str:
        return date
    y,m,d = date[0:4],date[4:6],date[6:8]
    date_Str = f'{y}-{m}-{d}'
    return pd.to_datetime(date_Str, format="%Y-%m-%d")

rel_Dat['PO Document Date'] = rel_Dat.apply(data_changer, axis=1) ### Fix dates
rel_Dat['PO Document Date'] = rel_Dat['PO Document Date'].dt.strftime('%Y-%m-%d') ### convert to strings for sql lite

In [4]:
nums_lines = rel_Dat[['PO Number','PO Line Number']].sort_values('PO Number')
numbers_lines = set(list(zip(nums_lines['PO Number'].to_list(), nums_lines['PO Line Number'].to_list())))
clean_frame_rel_dat = pd.DataFrame(dict(zip(rel_Dat.columns, [[]*len(rel_Dat.columns)])))
for item in tqdm.tqdm(numbers_lines): #PO Numbers
    ponum = item[0]
    polin = item[1]
    check = rel_Dat[(rel_Dat['PO Number']==ponum)&(rel_Dat['PO Line Number']==polin)]
    cols = [
        'Changed On Date',
        'Confirmed Delivery Date',
        'Creation Date of Confirmation',
        'Posting Date',
        'Scheduled-relevant delivery date'
    ]
    check[cols] = check[cols].replace({'0':np.nan, 0:np.nan})
    for col in cols:
        if col == 'Scheduled-relevant delivery date':
            check[col] = pd.to_datetime(check[col].astype(str), format='%m/%d/%Y')
        else:
            check[col] = pd.to_datetime(check[col].astype(str), format='%Y%m%d')
    last_changed, error_final = check.sort_values(by=['Changed On Date'], ascending=False).iloc[0]['Changed On Date'], check.sort_values(by=['Changed On Date'], ascending=False).iloc[0]['Supply Chain Error Reason Code']
    last_due_date = check.sort_values(by=['Scheduled-relevant delivery date'], ascending=False).iloc[0]['Scheduled-relevant delivery date']
    arrival = check.sort_values(by=['Posting Date'], ascending=False).iloc[0]['Posting Date']
    last = check.groupby(['PO Number','PO Line Number']).first()
    new_row = pd.DataFrame({
        'Zsupplier___T': last['Zsupplier___T'],
        'Vendor Number': last['Vendor Number'],
        'Purchasing organization': last['Purchasing organization'],
        'PO Document Date': last['PO Document Date'],
        'PO Number': ponum,
        'PO Line Number': polin,
        'Material': last['Material'],
        'Material Group': last['Material Group'],
        'MRP controller': last['MRP controller'],
        'Changed On Date': last_changed,
        'Confirmed Delivery Date': last['Confirmed Delivery Date'],
        'Creation Date of Confirmation': last['Creation Date of Confirmation'],
        'Posting Date': arrival,
        'Item Delivery Date': last['Item Delivery Date'],
        'Scheduled-relevant delivery date': last_due_date,
        'Supply Chain Error Reason Code': error_final
    })
    clean_frame_rel_dat = pd.concat([clean_frame_rel_dat, new_row])

100%|█████████████████████████████████████| 67284/67284 [26:08<00:00, 42.91it/s]


create SQL Lite table

In [5]:
reliability_data_table ="""
Supplier Text,
Vendor_Number Integer,
Purchasing_Organization Integer,
PO_Document_Date Text,
PO_Number Integer,
PO_Line_Number Integer,
Material Text,
Material_Group Text,
MRP_controller Text,
Changed_On_Date Text,
Confirmed_Delivery_Date Text,
Creation_Date_of_Confirmation Text,
Posting_Date Text,
Item_Delivery_Date Text,
Scheduled_relevant_delivery_date Text,
Error_Code Text
""" # Create structure of new table
insert_structure = clean_frame_rel_dat[[
    'Zsupplier___T',
    'Vendor Number',
    'Purchasing organization',
    'PO Document Date',
    'PO Number',
    'PO Line Number',
    'Material',
    'Material Group',
    'MRP controller',
    'Changed On Date',
    'Confirmed Delivery Date',
    'Creation Date of Confirmation',
    'Posting Date',
    'Item Delivery Date',
    'Scheduled-relevant delivery date',
    'Supply Chain Error Reason Code'
          ]] #grab data in correct order
insert_structure = insert_structure.set_axis(['Supplier' ,
'Vendor_Number' ,
'Purchasing_Organization' ,
'PO_Document_Date' ,
'PO_Number' ,
'PO_Line_Number' ,
'Material' ,
'Material_Group' ,
'MRP_controller' ,
'Changed_On_Date' ,
'Confirmed_Delivery_Date' ,
'Creation_Date_of_Confirmation' ,
'Posting_Date' ,
'Item_Delivery_Date' ,
'Scheduled_relevant_delivery_date',
'Error_Code'
                                            ], axis=1, inplace=False) #rename pd data to match sql table
conn = sqlite3.connect('test_database')
c = conn.cursor()
c.execute(f"DROP TABLE IF EXISTS reliability_table")
c.execute(f"CREATE TABLE IF NOT EXISTS reliability_table ({reliability_data_table})")
insert_structure.to_sql('reliability_table', conn, if_exists='append', index = False)
conn.commit()
conn.close()

### 2. Schedule A Data

In [6]:
scd_Dat = pd.read_excel('data_house.xlsx', sheet_name='Schedule A', header = 3) #read in data
scd_Dat = scd_Dat.rename(columns={'Unnamed: 4': 'Material_Description', 'Unnamed: 7': 'Plant_Name', 'Unnamed: 9': 'Material_Group_Description',  'Unnamed: 11': 'Supply_Chain_Lead'}) ### Enter in new header names

In [7]:
scd_Dat['Minimum Qty'] = pd.to_numeric(scd_Dat['Minimum Qty'], errors='coerce').fillna(0)
scd_Dat['PPU'] = scd_Dat['Current\nPrice'] / scd_Dat['Price Unit']
scd_Dat['Order_Price'] = scd_Dat['PPU'] * scd_Dat['Minimum Qty']

create SQL Lite Table

In [8]:
schedule_data_table ="""
Vendor_Number Integer,
Vendor_Name Text,
Purchasing_org Integer,
Material Text,
Material_Name,
Specials_Indicator Text,
Plant Integer,
Plant_Name Text,
Material_Group Text,
Material_Group_Name Text,
Purchasing_group Text,
Supply_Chain_Lead Text,
Plan_Delivery_Time_Days Integer,
Minimum_Qty Integer,
Overdelivery_Tolerance Text,
Underdelivery_Tolerance Text,
Base_Unit Text,
Scales_Indicator  Text,
MM_Deletion_flag Text,
Current_Price Real,
Price_Unit Real,
PPU Real,
Order_Price Real
""" # Create structure of new table
insert_structure_scd = scd_Dat[
    ['Vendor Number', 'Vendor Name', 'Purchasing org.', 'Material',
       'Material_Description', 'Specials Indicator', 'Plant', 'Plant_Name',
       'Material Group', 'Material_Group_Description', 'Purchasing group',
       'Supply_Chain_Lead', 'Plan Delivery Time (Days)', 'Minimum Qty',
       'Overdelivery Tolerance', 'Underdelivery Tolerance', 'Base Unit',
       'Scales Indicator', 'MM Deletion flag', 'Current\nPrice', 'Price Unit',
       'PPU', 'Order_Price']
] #grab data in correct order
insert_structure_scd = insert_structure_scd.set_axis([
'Vendor_Number',
'Vendor_Name',
'Purchasing_org',
'Material',
'Material_Name',
'Specials_Indicator',
'Plant',
'Plant_Name',
'Material_Group',
'Material_Group_Name',
'Purchasing_group',
'Supply_Chain_Lead',
'Plan_Delivery_Time_Days',
'Minimum_Qty',
'Overdelivery_Tolerance',
'Underdelivery_Tolerance',
'Base_Unit',
'Scales_Indicator',
'MM_Deletion_flag',
'Current_Price',
'Price_Unit',
'PPU',
'Order_Price'], axis=1, inplace=False) #rename pd data to match sql table
conn = sqlite3.connect('test_database')
c = conn.cursor()
c.execute(f"DROP TABLE IF EXISTS schedule_table")
c.execute(f"CREATE TABLE IF NOT EXISTS schedule_table ({schedule_data_table})")
insert_structure_scd.to_sql('schedule_table', conn, if_exists='append', index = False)
conn.commit()
conn.close()

### 3. Series 1 Component Data

In [9]:
comp_Dat = pd.read_excel('data_house.xlsx', sheet_name='Series 1 Component List', header = 0) #read in data

Create SQL Lite Table

In [10]:
components_data_table ="""
Part_Number	Text,
Part_Desciption	Text,
Series_1	Text,
Supplier 	Text
""" # Create structure of new table
insert_structure_comp = comp_Dat.set_axis([
"Part_Number",
"Part_Desciption",
"Series_1",
"Supplier"], axis=1, inplace=False) #rename pd data to match sql table
conn = sqlite3.connect('test_database')
c = conn.cursor()
c.execute(f"DROP TABLE IF EXISTS components_table")
c.execute(f"CREATE TABLE IF NOT EXISTS components_table ({components_data_table})")
insert_structure_comp.to_sql('components_table', conn, if_exists='append', index = False)
conn.commit()
conn.close()